In [1]:
# 種々のdatasetをintagrateします

In [2]:
from datasets import load_dataset
import json
import os
from tqdm import tqdm
import yaml

with open('config.yaml', 'r') as file:
    conf= yaml.safe_load(file)
output_path=conf["output_path"]
max_records=conf["max_records"]
print(conf)

/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'output_path': '../../data/integrated_text.jsonl', 'max_records': 1000000}


In [3]:
#ここでは例として､日英のwikipediaを読み込んでみます

#TODO: よく考えると､これだとRAMが足りなくなるかも?
dataset_list = [
    load_dataset("hpprc/wikipedia-20240101", split="train").shuffle(), #日本語
    load_dataset("wikipedia", "20220301.en",split="train").shuffle(), #英語
]

/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
#datasetの中身を確認してみます｡
#各recordは辞書形式になっています｡大切なのは､ textです｡
for data in dataset_list[0]:
    break
print(data)

{'id': '1739738', 'url': 'https://ja.wikipedia.org/wiki/%E5%B8%AF%E7%B4%94%E4%B9%9F', 'title': '帯純也', 'text': "帯 純也（おび じゅんや、1972年 - ）は、TBSテレビ制作局所属のテレビプロデューサー。\n\n来歴 \n立命館大学卒業、早稲田大学大学院政治学研究科修士課程修了。大学卒業後にTBS入社以来、制作畑を歩む。途中ドラマ制作を経て制作局バラエティ制作センター（旧制作三部）所属。\n\n過去には『さんまのスーパーからくりTV』『稲垣吾郎のマイフェアレディ』、木梨憲武の『お茶の水ハカセ』『中居正広の金曜日のスマたちへ』の総合演出・プロデューサーを担当。\n\n過去の担当番組 \n3年B組金八先生（第5シリーズ、演出補）\nCOUNT DOWN TV（ディレクター）\nPOP FILE\nUSO!?ジャパン（ディレクター）\nさんま・玉緒のお年玉あんたの夢をかなえたろかスペシャル\n明石家さんまつり\nオールスター感謝祭（1996年秋、1999年春フロアディレクター）\nTHE CHAIR\nGoro's Bar\nドリーム・プレス社\nバラエティーニュース キミハ・ブレイク\nGoroプレゼンツ マイ・フェア・レディ\n科学バラエティ お茶の水ハカセ\nカミスン!\nさんまのSUPERからくりTV（総合演出→プロデューサー）\nライブB♪（プロデューサー）\n直撃!コロシアム!! ズバッと!TV（プロデューサー）\n中居正広の金曜日のスマイルたちへ（チーフディレクター→プロデューサー）\n音楽の日（制作プロデューサー→協力プロデューサー）\n\n外部リンク \n社員紹介:制作センター|TBSテレビ:採用2013（インターネットアーカイブ）\n\nTBSの人物\n日本のテレビプロデューサー\n早稲田大学出身の人物\n立命館大学出身の人物\n新潟明訓高等学校出身の人物\n新潟市出身の人物\n1972年生\n存命人物"}


In [5]:
# 各datasetをmergeして､一つのjsonlにまとめます｡
# ファイルは output_pathに生成されます
# 各datasetは予めクリーニング, dedupされている必要があります｡
#TODO: BTMのため､ジャンル別にデータを並べ替えたい

if os.path.exists(output_path):
    print(f"jsonl file {output_path} already exists. Overwrite? (y/n)")
    ans=input()

    if ans=="y" or "Y":
        overwrite=True
        print("overwriting files...")
        with open(output_path,"w") as f:
            f.write("")
    else:
        overwrite=False
        print("aborted.")
if overwrite:
    with open(output_path,"a") as f:
        for dataset in dataset_list:
            print(dataset)
            cnt=0
            for data in tqdm(dataset):
                out_text=json.dumps({"text":data["text"]}, ensure_ascii=False)
                #jsonlで書き出し

                f.write(out_text+"\n") 

                cnt+=1
                if cnt>max_records:
                    break

jsonl file ../../data/integrated_text.jsonl already exists. Overwrite? (y/n)
overwriting files...
Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1395760
})


 72%|███████▏  | 1000000/1395760 [01:01<00:24, 16173.56it/s]


Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 6458670
})


 15%|█▌        | 1000000/6458670 [01:06<06:04, 14960.24it/s]


In [6]:
dataset

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 6458670
})